## Import necessary modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import the data

In [2]:
bom = pd.read_csv("data/bom.movie_gross.csv.gz")
imdb_names = pd.read_csv("data/imdb.name.basics.csv.gz")
# imdb_title_akas = pd.read_csv("data/imdb.title.akas.csv.gz")
imdb_title_basics = pd.read_csv("data/imdb.title.basics.csv.gz")
# imdb_title_princ = pd.read_csv("data/imdb.title.principals.csv.gz")
imdb_title_ratings = pd.read_csv("data/imdb.title.ratings.csv.gz")
tmdb = pd.read_csv("data/tmdb.movies.csv.gz")
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv.gz', sep='\t')
rt_reviews = pd.read_csv("data/rt.reviews.tsv.gz", sep='\t', encoding='latin-1')
tn_movie_budgets = pd.read_csv("data/tn.movie_budgets.csv.gz", sep=',')

In [41]:
inner_bom_tmdb = bom.merge(tmdb, left_on='title', right_on='title')

In [45]:
imdb_title_basics = pd.read_csv("data/imdb.title.basics.csv.gz")

In [248]:
brad_pitt = imdb_names[imdb_names['primary_name'] == 'Brad Pitt'].copy()
brad_pitt2 = brad_pitt.copy()

In [249]:
brad_pitt

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
24026,nm0000093,Brad Pitt,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711"


In [56]:
brad_pitt[['movie_1', 'movie_2', 'movie_3', 'movie_4']] = brad_pitt.apply(lambda x: pd.Series(x['known_for_titles'].split(',')), axis=1)

In [55]:
brad_pitt.apply(lambda x: pd.Series(x['known_for_titles'].split(',')), axis=1).stack()

24026  0    tt1210166
       1    tt0114746
       2    tt0356910
       3    tt0816711
dtype: object

In [60]:
brad_pitt_melt = brad_pitt.melt(id_vars='primary_name', value_vars=['movie_1', 'movie_2', 'movie_3', 'movie_4'])

In [61]:
brad_pitt_melt.merge(brad_pitt, left_on='primary_name', right_on='primary_name')

,primary_name,variable,value,nconst,birth_year,death_year,primary_profession,known_for_titles,kft,movie_1,movie_2,movie_3,movie_4
0,Brad Pitt,movie_1,tt1210166,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711
1,Brad Pitt,movie_2,tt0114746,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711
2,Brad Pitt,movie_3,tt0356910,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711
3,Brad Pitt,movie_4,tt0816711,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711


In [62]:
brad_pitt_melt

,primary_name,variable,value
0,Brad Pitt,movie_1,tt1210166
1,Brad Pitt,movie_2,tt0114746
2,Brad Pitt,movie_3,tt0356910
3,Brad Pitt,movie_4,tt0816711


In [97]:
imdb_head = imdb_title_basics.head().copy()

In [98]:
imdb_head['genres_list'] = imdb_head.genres.str.split(',')

In [121]:
s = imdb_head.apply(lambda x: pd.Series(x['genres_list']), axis=1).stack().reset_index(level=1, drop=True)

In [125]:
s.name = 'Genres'
imdb_genres = imdb_head.join(s).copy()

In [133]:
genres_dummies = pd.get_dummies(imdb_genres.Genres).sum(level=0)

In [136]:
imdb_head.join(genres_dummies).drop(['genres', 'genres_list'], axis=1)

,tconst,primary_title,original_title,start_year,runtime_minutes,Action,Biography,Comedy,Crime,Drama,Fantasy
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,1,0,0,1,1,0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,0,1,0,0,1,0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,0,0,0,0,1,0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,0,0,1,0,1,0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,0,0,1,0,1,1


### One-Hot Encode Genres for Every Movie

In [140]:
def OHE_genres(movie_list):
    movie_list['genres_list'] = movie_list.genres.str.split(',')
    s = movie_list.apply(lambda x: pd.Series(x['genres_list']), 
                                             axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'Genres'
    imdb_genres = movie_list.join(s).copy()
    genres_dummies = pd.get_dummies(imdb_genres.Genres).sum(level=0)
    ml = movie_list.join(genres_dummies).drop(['genres', 'genres_list'], axis=1)
    return ml

In [77]:
inner_bom_imdb = bom.merge(imdb_title_basics, left_on='title', right_on='primary_title')

In [81]:
inner_bom_imdb.set_index('tconst', inplace=True)

In [142]:
inner_bom_imdb = OHE_genres(inner_bom_imdb)

### Fixing Movie Gross Problems

In [151]:
inner_bom_imdb.loc[inner_bom_imdb.foreign_gross == '1,019.4','foreign_gross'] = 1_019_440_000
inner_bom_imdb.loc[inner_bom_imdb.foreign_gross == '1,010.0'] = 1_010_000_000
inner_bom_imdb.loc[inner_bom_imdb.foreign_gross == '1,369.5','foreign_gross'] = 1_369_500_000
inner_bom_imdb.loc[inner_bom_imdb.foreign_gross == '1,163.0','foreign_gross'] = 1_163_000_000

In [166]:
inner_bom_imdb['total_gross'] = inner_bom_imdb['domestic_gross'] + inner_bom_imdb.foreign_gross.astype(float).fillna(0)

In [ ]:
null_box_office = inner_bom_imdb[inner_bom_imdb.domestic_gross.isna()].index

In [177]:
inner_bom_imdb = inner_bom_imdb.drop(null_box_office)

### Making Each Movie in names a row

In [259]:
names = imdb_names.copy()
names2 = imdb_names.copy()

In [250]:
brad_pitt['kft'] = brad_pitt.known_for_titles.str.split(',')
s = brad_pitt.apply(lambda x: pd.Series(x['kft']), axis=1).stack().reset_index(level=1, drop=True)
s.name = "Known_Movies"
brad_pitt_movies = brad_pitt.join(s).drop(['nconst', 'birth_year',
                                           'death_year', 
                                           'known_for_titles', 'kft', 'primary_profession'], axis=1)

In [260]:
names['kft'] = names.known_for_titles.str.split(',')
s = names.apply(lambda x: pd.Series(x['kft']), axis=1).stack().reset_index(level=1, drop=True)
s.name = "Known_Movies"
names_movies = names.join(s).drop(['nconst', 'birth_year',
                                           'death_year', 
                                           'known_for_titles', 'kft', 'primary_profession'], axis=1)

In [268]:
names_movies.set_index('Known_Movies', inplace=True)

In [256]:
brad_pitt2['profs'] = brad_pitt2.primary_profession.str.split(',')
s = brad_pitt2.apply(lambda x: pd.Series(x['profs']), 
                                         axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Professions'
brad_pitt_profs = brad_pitt2.join(s).copy()
profs_dummies = pd.get_dummies(brad_pitt_profs.Professions).sum(level=0)
brad_pitt_profs = brad_pitt_profs.join(profs_dummies)
brad_pitt_profs.drop(['nconst', 'birth_year', 'death_year', 'known_for_titles',
                      'primary_profession', 'Professions', 'profs'], axis=1, inplace=True)
brad_pitt_profs.drop_duplicates(inplace=True)

In [261]:
names2['profs'] = names2.primary_profession.str.split(',')
s = names2.apply(lambda x: pd.Series(x['profs']), 
                                         axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Professions'
names_profs = names2.join(s).copy()
profs_dummies = pd.get_dummies(names_profs.Professions).sum(level=0)
names_profs = names_profs.join(profs_dummies)
names_profs.drop(['nconst', 'birth_year', 'death_year', 'known_for_titles',
                      'primary_profession', 'Professions', 'profs'], axis=1, inplace=True)
names_profs.drop_duplicates(inplace=True)

In [264]:
names_profs.shape

(602373, 38)

In [257]:
brad_pitt_profs

,primary_name,actor,producer,soundtrack
24026,Brad Pitt,1,1,1


### Merge People and Movies Dataframe

In [266]:
imdb_title_basics.set_index('tconst', inplace=True)

In [272]:
titles_people_merge = inner_bom_imdb.merge(names_movies, left_index=True, right_index=True )

In [274]:
titles_people_merge.shape

(60388, 33)

In [276]:
titles_people_profs_merge = titles_people_merge.merge(names_profs, left_on='primary_name', right_on='primary_name')

In [277]:
titles_people_profs_merge.shape

(67362, 70)

In [280]:
titles_people_profs_merge.head()

,title,studio,domestic_gross,foreign_gross,year,primary_title,original_title,start_year,runtime_minutes,Action,...,publicist,set_decorator,sound_department,soundtrack,special_effects,stunts,talent_agent,transportation_department,visual_effects,writer
0,Wazir,Relbig.,1100000.0,NaN,2016,Wazir,Wazir,2016,103.0,1,...,0,0,0,0,0,0,0,0,0,0
1,Wazir,Relbig.,1100000.0,NaN,2016,Wazir,Wazir,2016,103.0,1,...,0,0,0,0,0,0,0,0,0,0
2,Wazir,Relbig.,1100000.0,NaN,2016,Wazir,Wazir,2016,103.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Wazir,Relbig.,1100000.0,NaN,2016,Wazir,Wazir,2016,103.0,1,...,0,0,0,0,0,0,0,0,0,0
4,Wazir,Relbig.,1100000.0,NaN,2016,Wazir,Wazir,2016,103.0,1,...,0,0,0,0,0,0,0,0,0,0


In [281]:
titles_people_profs_merge.to_csv('data/title_people_profs_merge.csv')

In [282]:
tppm = titles_people_profs_merge.copy()